<a href="https://colab.research.google.com/github/neennera/AI_pitch/blob/main/Pre_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import os
from torch import nn
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
from torch.utils.data import Dataset

import pandas as pd
import torchaudio
import pickle
import numpy as np

# DataLoader

In [3]:
#import zipfile
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
'''!pip install patool
!pip install pyunpack
from pyunpack import Archive'''
Archive("/content/gdrive/MyDrive/AI/processed.rar").extractall("/content/gdrive/MyDrive/AI/DATA")

In [4]:
train_feat_path="/content/gdrive/MyDrive/AI/DATA/processed/train/"
test_feat_path="/content/gdrive/MyDrive/AI/DATA/processed/test/"

In [20]:
classmap=["C","C#","D","D#","E","F","F#","G","G#","A","A#","B"]
df = pd.read_csv("/content/gdrive/MyDrive/AI/labels.csv")

In [7]:
train_name=os.listdir(train_feat_path)
test_name=os.listdir(test_feat_path)

class CustomTrainDataset(Dataset):
    def __init__(self, feat_path=train_feat_path, transform=None, target_transform=None):
        self.img_labels = os.listdir(train_feat_path)
        self.feat_path = feat_path 

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        data = (torch.from_numpy(np.load(self.feat_path +self.img_labels[idx])))
        #data = data[0]
        #label = int(self.img_labels[idx][0:2])
        label = classmap.index(df["label"][idx])
        return data, label

class CustomTestDataset(Dataset):
    def __init__(self, feat_path=test_feat_path , transform=None, target_transform=None):
        self.img_labels = os.listdir(test_feat_path)
        self.feat_path = feat_path 

    def __len__(self):
      return len(self.img_labels)

    def __getitem__(self, idx):
        data = (torch.from_numpy(np.load(self.feat_path +self.img_labels[idx])))
        #data = data[0]
        label = classmap.index(df["label"][idx])
        return data, label

train_dataset=CustomTrainDataset()
test_dataset=CustomTestDataset()

In [7]:
'''classmap = [
      'A0', 'A#0', 'B0', 'C1', 'C#1', 'D1', 'D#1', 'E1', 'F1', 'F#1', 'G1', 'G#1', 'A1', 'A#1', 
     'B1', 'C2', 'C#2', 'D2', 'D#2', 'E2', 'F2', 'F#2', 'G2', 'G#2', 'A2', 'A#2', 'B2', 'C3', 
     'C#3', 'D3', 'D#3', 'E3', 'F3', 'F#3', 'G3', 'G#3', 'A3', 'A#3', 'B3', 'C4', 'C#4', 'D4', 
     'D#4', 'E4', 'F4', 'F#4', 'G4', 'G#4', 'A4', 'A#4', 'B4', 'C5', 'C#5', 'D5', 'D#5', 'E5', 
     'F5', 'F#5', 'G5', 'G#5', 'A5', 'A#5', 'B5', 'C6', 'C#6', 'D6', 'D#6', 'E6', 'F6', 'F#6', 
     'G6', 'G#6', 'A6', 'A#6', 'B6', 'C7', 'C#7', 'D7', 'D#7', 'E7', 'F7', 'F#7', 'G7', 'G#7', 
     'A7', 'A#7', 'B7', 'C8', 'C#8', 'D8', 'D#8', 'E8', 'F8', 'F#8', 'G8', 'G#8', 'A8', 'A#8', 
     'B8', 'C9', 'C#9', 'D9', 'D#9', 'E9', 'F9', 'F#9', 'G9']'''

#Model

In [22]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        # 4 conv blocks / flatten / linear / softmax
        self.conv1 = nn.Sequential(
            nn.Conv2d(
                in_channels=1,
                out_channels=4,
                kernel_size=3,
                stride=1,
                padding='same'
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(
                in_channels=4,
                out_channels=16,
                kernel_size=3,
                stride=1,
                padding='same'
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(
                in_channels=16,
                out_channels=32,
                kernel_size=3,
                stride=1,
                padding='same'
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.flatten = nn.Flatten()
        self.linear = nn.Linear(32*31*6, 12)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, input_data):
        x = self.conv1(input_data)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.flatten(x)
        logits = self.linear(x)
        predictions = self.softmax(logits)
        return predictions

In [39]:
from sklearn.metrics import confusion_matrix, precision_score,accuracy_score
loss_fn = nn.MSELoss()
loss_fn = nn.CrossEntropyLoss()
def M_loss_func(pred, target):
  """k = len(target)
  targets = torch.from_numpy(np.zeros((k,107), dtype="float32"))
  for i in range(k) :
    targets[i][target[i]] = 1.0"""
  return loss_fn(pred,target)

def M_accuracy(pred,target):
  #tn, fp, fn, tp = confusion_matrix(y_true, y_pred_class).ravel()
  return accuracy_score(target, preds)
  #return precision_score(target, preds,average='micro')

In [10]:
model = Model()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), 3e-4)

checkpoint = { 
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict(),
}

def save(train_losses,metrics,i):
    loss_log=[]
    acc_log=[]
    baseline_log=[]
    
    path='/content/gdrive/MyDrive/AI/weight/H'+str(i)+'/checkpoint.pth'
    path_log='/content/gdrive/MyDrive/AI/weight/H'+str(i)
    path_old='/content/gdrive/MyDrive/AI/weight/H'+str(i-1)
    torch.save(checkpoint, path)

    with open(path_old+'/loss_log.pkl', 'rb') as f:
        loss_log = pickle.load(f)
    with open(path_old+'/acc_log.pkl', 'rb') as f:
        acc_log = pickle.load(f)        

    for k in train_losses :
        loss_log.append(k)
    acc_log = acc_log + metrics

    with open(path_log+'/loss_log.pkl', 'wb') as f:
        pickle.dump(loss_log, f)
    with open(path_log+'/acc_log.pkl', 'wb') as f:
        pickle.dump(acc_log, f)

# Training loop

In [11]:
from tqdm.auto import tqdm

def get_epoch(k, train_dataloader, val_dataloader, batch_size,st,ed):
    print("epoch","\t train loss \t accuracy \t\t val loss")
    train_loss ,train_acc, val_loss=0,0,0
    for i in range(st,ed+1) :
        train_epoch(i, train_dataloader, val_dataloader, batch_size)
        #torch.save(model.state_dict(), "weight/convo2/mymodel.pt")

def train_epoch(k, train_dataloader, val_dataloader, batch_size):
    train_batch_loss = []
    train_batch_acc = []
    for batch in tqdm(train_dataloader):
        xb, yb = batch

        optimizer.zero_grad()
        yhatb = model(xb)
        yhatb()=  yhatb.argmax(dim=1)

        loss = M_loss_func(yhatb, yb)
        acc = M_accuracy(yhatb, yb)
        loss.backward()
        optimizer.step()

        train_batch_loss.append(loss.detach().cpu().numpy())
        train_batch_acc.append(acc)

    train_loss = np.mean(train_batch_loss)  # average train loss among all batches
    train_acc = np.mean(train_batch_acc)  # average acc among all batches     

    val_batch_loss = []
    losses=[]
    metrics=[]
    for batch in tqdm(val_dataloader):
        x, y = batch
        x = x.to(device)
        y = y.to(device)

        # we don't need to calculate gradients when evaluating
        with torch.no_grad():
            yhatb = model(x)  # forward propagation
            loss = M_loss_func(yhatb, y)  # compute val loss
            acc = M_accuracy(yhatb, y)
            losses.append(loss.detach().cpu())
            metrics.append(acc)
        val_batch_loss.append(loss.cpu().numpy())
    val_loss = np.mean(val_batch_loss)
    if(k>0):
      save(losses,metrics,k)
    print(k, "\t", train_loss , "\t", train_acc,"\t", val_loss, "\t" , yhatb.mean())

In [12]:
batch_size = 128
epochs = 10
lr = 0.1
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(test_dataset, batch_size=batch_size)
for batch in tqdm(train_dataloader):
    xb, yb = batch
    break

In [41]:
losses=[]
metrics=[]
model=Model()
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(test_dataset, batch_size=batch_size)
get_epoch(15, train_dataloader, val_dataloader, batch_size,0,15)

epoch 	 train loss 	 accuracy 		 val loss


  0%|          | 0/168 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

# Graph

In [ ]:
loss_log=[]
acc_log=[]
i=16

path='/content/gdrive/MyDrive/AI/weight/H'+str(i)+'/checkpoint.pth'
path_log='/content/gdrive/MyDrive/AI/weight/H'+str(i)
path_old='/content/gdrive/MyDrive/AI/weight/H'+str(i-1)
torch.save(checkpoint, path)

with open(path_old+'/loss_log.pkl', 'rb') as f:
    loss_log = pickle.load(f)
with open(path_old+'/acc_log.pkl', 'rb') as f:
    acc_log = pickle.load(f)

In [ ]:
import matplotlib.pyplot as plt
t = torch.arange(len(loss_log)).numpy()
fig,ax = plt.subplots(2,1,figsize=(9, 9))
ax[0].plot(t, loss_log, label="Loss")
ax[1].plot(t, acc_log, label="Acc")

# Predict



In [ ]:
def predict(model, input, target, class_mapping) :
  model.eval() #เป็นสวิตท์ model.eval() ใช่ในการทาย // model.train() ใช้ในการเทรน
  
  with torch.no_grad(): #ใช้ model แบบไม่คำนวน grad
    pred = model(input)
    #จะ map ไปเป็น class โดย pred อยู่ในรูป (batch, number , class) -> (1,1,10)
    pred_index = (pred.argmax(dim=1)).to(dtype=int)
        # pred[0] เอามาแค่ตัวแรก
        # argmax(0)  เลือกเอาอากิวเมนต์ที่มีค่าสูงสุด [เพราะ softmax ทำนายออกมาเป็น % ความน่าจะเป็น]
    preds=[]
    expected=[]
    for i in range(len(pred_index)) :
      preds.append(class_mapping[pred_index[i]])
      expected.append(class_mapping[target[i]])
  return preds, expected

In [ ]:
from sys import breakpointhook
'''classmap = [
    'A0', 'A#0', 'B0', 'C1', 'C#1', 'D1', 'D#1', 'E1', 'F1', 'F#1', 'G1', 'G#1', 'A1', 'A#1', 
     'B1', 'C2', 'C#2', 'D2', 'D#2', 'E2', 'F2', 'F#2', 'G2', 'G#2', 'A2', 'A#2', 'B2', 'C3', 
     'C#3', 'D3', 'D#3', 'E3', 'F3', 'F#3', 'G3', 'G#3', 'A3', 'A#3', 'B3', 'C4', 'C#4', 'D4', 
     'D#4', 'E4', 'F4', 'F#4', 'G4', 'G#4', 'A4', 'A#4', 'B4', 'C5', 'C#5', 'D5', 'D#5', 'E5', 
     'F5', 'F#5', 'G5', 'G#5', 'A5', 'A#5', 'B5', 'C6', 'C#6', 'D6', 'D#6', 'E6', 'F6', 'F#6', 
     'G6', 'G#6', 'A6', 'A#6', 'B6', 'C7', 'C#7', 'D7', 'D#7', 'E7', 'F7', 'F#7', 'G7', 'G#7', 
     'A7', 'A#7', 'B7', 'C8', 'C#8', 'D8', 'D#8', 'E8', 'F8', 'F#8', 'G8', 'G#8', 'A8', 'A#8', 
     'B8', 'C9', 'C#9', 'D9', 'D#9', 'E9', 'F9', 'F#9', 'G9']'''

validation=DataLoader(test_dataset, batch_size=batch_size)
for batch in tqdm(train_dataloader):
    xb, yb = batch
    break
  
predicted, expected = predict(model, xb, yb, classmap)
for i in range(20) :
    print(f"Predicted: '{predicted[i]}', expected: '{expected[i]}'")